In [133]:
import copy
import numpy as np
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, Data2D,build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (ProductOvernightCapFloorlet, ProductOvernightSwaption, ProductOvernightCapFloor)
import QuantLib as ql
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor
from fixedincomelib.utilities.risk_reporting import createValueReport, risk_vectors_to_df
from fixedincomelib.sabr import valuation_engine_sabr, SabrModel
from fixedincomelib.analytics import SABRCalculator


In [134]:
IndexManager.instance()

In [135]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [136]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [137]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]

In [138]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [139]:
# Corr surface
corr_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]
corr_tenors   = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 30.0]

def corr_surface(expiry, tenor):
    base = 0.86
    decay_e = 0.08*np.log(1.0 + 2.0*expiry)
    decay_t = 0.10*np.log(1.0 + tenor)
    corr = base - decay_e - decay_t
    return float(np.clip(corr, 0.50, 0.90))

rows = []
for e in corr_expiries:
    for t in corr_tenors:
        rows.append([float(e), float(t), corr_surface(e, t)])

df_corr = pd.DataFrame(rows, columns=["EXPIRY","TENOR","CORR"])
df_corr["INDEX"] = "SOFR-1B"

corr_pivot = (df_corr.pivot(index="EXPIRY", columns="TENOR", values="CORR").sort_index(axis=0).sort_index(axis=1))

bu_objs = []
bu_objs.append(Data2D.createDataObject("corr", "SOFR-1B", corr_pivot))


In [140]:
one_bd = 1.0/252.0
sabr_expiries_1bd = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]

def normalvol_1bd(expiry):
    return float(0.0095*np.exp(-0.22*expiry) + 0.0026)

def nu_1bd(expiry):
    return float(0.85*np.exp(-0.35*expiry) + 0.20)

def rho_1bd(expiry):
    return float(np.clip(-0.42 + 0.06*np.log(1.0 + expiry), -0.60, -0.05))

beta_fixed = 0.60

rows = []
for e in sabr_expiries_1bd:
    rows.append([
        "SOFR-1B",
        float(e),
        float(one_bd),
        normalvol_1bd(e),
        float(beta_fixed),
        nu_1bd(e),
        rho_1bd(e),
    ])

sabr_sofr_1bd = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

for idx_name, sub in sabr_sofr_1bd.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        bu_objs.append(
            Data2D.createDataObject(
                data_type=param.lower(),
                data_convention=idx_name,
                df=pivot
            )
        )


In [141]:
value_date = "2025-09-24"

bottom_up_build_methods_sofr = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr.append({
        "TARGET":          "SOFR-1B",
        "VALUES":          param,
        "INTERPOLATION":   "LINEAR",
        "SHIFT":           0.0,
        "VOL_DECAY_SPEED": 0.2
    })

bu_dc = DataCollection(bu_objs)

bottom_up_sofr_sabr = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=bu_dc,
    buildMethodCollection=bottom_up_build_methods_sofr,
    ycModel=yc
)

print("Bottom-up 1BD-tenor SABR components:", bottom_up_sofr_sabr.components.keys())

Bottom-up 1BD-tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [142]:
np.set_printoptions(threshold=np.inf, linewidth=200, suppress=True)
bottom_up_sofr_sabr.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,  -98.924743  ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0. 

In [143]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr}

ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr,
    vp,
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)

ve_bu_cap.calculateFirstOrderRisk()
g = np.asarray(ve_bu_cap.firstOrderRisk, dtype=float)
print(g)

a= ve_bu_cap.value[1]

Overnight Caplet PV (bottom-up): ['USD', np.float64(5404.750094120622)]
[ -1366.20071824  -1351.18752353  -1381.21391294 223204.35560817  18390.18983291      0.              0.              0.              0.              0.              0.              0.
      0.              0.              0.              0.              0.              0.              0.              0.              0.              0.              0.              0.
      0.              0.              0.              0.           1908.37926682   8499.71269495     28.09778388      0.              0.              0.              0.             -8.8898942
    -31.218599       -0.06580806      0.              0.              0.              0.             17.9129415      79.48398712      0.26134809      0.              0.              0.
      0.             -9.15561076    -39.72914986     -0.12656321      0.              0.        ]


In [144]:
report = createValueReport(vp, bottom_up_sofr_sabr, caplet_bu, request="all", space="pv")
g = report["param_risk"]
print(g.shape, g)

(54,) [   13.66200718    13.65874181    14.10887324 -2303.88511774  -191.76144079    -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.
    -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.
     0.             0.          1908.37926682  8499.71269495    28.09778388     0.             0.             0.             0.            -8.8898942    -31.218599      -0.06580806     0.
     0.             0.             0.            17.9129415     79.48398712     0.26134809     0.             0.             0.             0.            -9.15561076   -39.72914986    -0.12656321
     0.             0.        ]


In [145]:
report_q = createValueReport(vp, bottom_up_sofr_sabr, caplet_bu, request="all", space="quote")
dq = report_q["quote_risk"]
print(dq.shape)
print(dq)

(54,)
[ -13.51510549  -13.37147475  -13.66891923 2209.45798556  182.09751429    0.            0.            0.            0.            0.            0.            0.           -0.           -0.
   -0.           -0.           -0.           -0.           -0.           -0.           -0.           -0.           -0.           -0.           -0.           -0.            0.            0.
 1908.37926682 8499.71269495   28.09778388    0.            0.            0.            0.           -8.8898942   -31.218599     -0.06580806    0.            0.            0.            0.
   17.9129415    79.48398712    0.26134809    0.            0.            0.            0.           -9.15561076  -39.72914986   -0.12656321    0.            0.        ]


In [146]:
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

In [147]:
df_report = risk_vectors_to_df(bottom_up_sofr_sabr,report_q, yc_index="SOFR-1B") 

df_report.head(5)

,block,index,param,expiry,tenor,node,pos,dPV_dModelParam,hedgeWeightPV,dPV_dQuote,abs_model,abs_weight,abs_quote
0,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M September 24th, 2025...",0,"-1,366.200718",13.662007,-13.515105,"1,366.200718",13.662007,13.515105
1,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 ...",1,"-1,351.187524",13.658742,-13.371475,"1,351.187524",13.658742,13.371475
2,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x J...",2,"-1,381.213913",14.108873,-13.668919,"1,381.213913",14.108873,13.668919
3,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M June 24th, 2026 x Se...",3,"223,204.355608","-2,303.885118","2,209.457986","223,204.355608","2,303.885118","2,209.457986"
4,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M September 24th, 2026...",4,"18,390.189833",-191.761441,182.097514,"18,390.189833",191.761441,182.097514


In [148]:
MARKET_DF2 = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.71],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [149]:
data_objs2, dc2 = build_yc_data_collection(MARKET_DF2)

In [150]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [151]:
yc2 = YieldCurve("2025-09-24", dc2, build_methods)
print("Curve components:", yc2.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [152]:
value_date = "2025-09-24"

bottom_up_build_methods_sofr = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr.append({
        "TARGET":          "SOFR-1B",
        "VALUES":          param,
        "INTERPOLATION":   "LINEAR",
        "SHIFT":           0.0,
        "VOL_DECAY_SPEED": 0.2
    })

bu_dc = DataCollection(bu_objs)

bottom_up_sofr_sabr2 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=bu_dc,
    buildMethodCollection=bottom_up_build_methods_sofr,
    ycModel=yc2
)

print("Bottom-up 1BD-tenor SABR components:", bottom_up_sofr_sabr.components.keys())

Bottom-up 1BD-tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [153]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr}

ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr2,
    vp,
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)

b = ve_bu_cap.value[1]

Overnight Caplet PV (bottom-up): ['USD', np.float64(5404.885248562322)]


In [154]:
(b-a)/0.01

np.float64(13.515444170025148)

In [155]:
# Corr surface
corr_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]
corr_tenors   = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 30.0]

def corr_surface(expiry, tenor):
    base = 0.86
    decay_e = 0.08*np.log(1.0 + 2.0*expiry)
    decay_t = 0.10*np.log(1.0 + tenor)
    corr = base - decay_e - decay_t
    return float(np.clip(corr, 0.50, 0.90))

rows = []
for e in corr_expiries:
    for t in corr_tenors:
        rows.append([float(e), float(t), corr_surface(e, t)])

df_corr = pd.DataFrame(rows, columns=["EXPIRY","TENOR","CORR"])
df_corr["INDEX"] = "SOFR-1B"

corr_pivot = (df_corr.pivot(index="EXPIRY", columns="TENOR", values="CORR").sort_index(axis=0).sort_index(axis=1))

bu_objs_2 = []
bu_objs_2.append(Data2D.createDataObject("corr", "SOFR-1B", corr_pivot))


In [156]:
one_bd = 1.0/252.0
sabr_expiries_1bd = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]

def normalvol_1bd(expiry):
    return float(0.0095*np.exp(-0.22*expiry) + 0.0026)

def nu_1bd(expiry):
    return float(0.85*np.exp(-0.35*expiry) + 0.20)

def rho_1bd(expiry):
    return float(np.clip(-0.42 + 0.06*np.log(1.0 + expiry), -0.60, -0.05))

beta_fixed = 0.60

rows = []
for e in sabr_expiries_1bd:
    rows.append([
        "SOFR-1B",
        float(e),
        float(one_bd),
        normalvol_1bd(e),
        float(beta_fixed),
        nu_1bd(e),
        rho_1bd(e),
    ])

sabr_sofr_1bd_2 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])


m = ((sabr_sofr_1bd_2["INDEX"] == "SOFR-1B") & np.isclose(sabr_sofr_1bd_2["AXIS1"], 1.0)) 

sabr_sofr_1bd_2.loc[m, "NORMALVOL"] += 0.0001

for idx_name, sub in sabr_sofr_1bd_2.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        bu_objs_2.append(
            Data2D.createDataObject(
                data_type=param.lower(),
                data_convention=idx_name,
                df=pivot
            )
        )

In [157]:
value_date = "2025-09-24"

bottom_up_build_methods_sofr_2 = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr_2.append({
        "TARGET":          "SOFR-1B",
        "VALUES":          param,
        "INTERPOLATION":   "LINEAR",
        "SHIFT":           0.0,
        "VOL_DECAY_SPEED": 0.2
    })

bu_dc_2 = DataCollection(bu_objs_2)

bottom_up_sofr_sabr_3 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=bu_dc_2,
    buildMethodCollection=bottom_up_build_methods_sofr_2,
    ycModel=yc
)

print("Bottom-up 1BD-tenor SABR components:", bottom_up_sofr_sabr_3.components.keys())

Bottom-up 1BD-tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [158]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr}

ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr_3,
    vp,
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)

c = ve_bu_cap.value[1]

Overnight Caplet PV (bottom-up): ['USD', np.float64(5405.606680442321)]


In [159]:
(c-a)/0.0001

np.float64(8565.863216990692)

In [160]:
# Corr surface
corr_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]
corr_tenors   = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 30.0]

def corr_surface(expiry, tenor):
    base = 0.86
    decay_e = 0.08*np.log(1.0 + 2.0*expiry)
    decay_t = 0.10*np.log(1.0 + tenor)
    corr = base - decay_e - decay_t
    return float(np.clip(corr, 0.50, 0.90))

rows = []
for e in corr_expiries:
    for t in corr_tenors:
        rows.append([float(e), float(t), corr_surface(e, t)])

df_corr = pd.DataFrame(rows, columns=["EXPIRY","TENOR","CORR"])
df_corr["INDEX"] = "SOFR-1B"

corr_pivot = (df_corr.pivot(index="EXPIRY", columns="TENOR", values="CORR").sort_index(axis=0).sort_index(axis=1))

bu_objs_3 = []
bu_objs_3.append(Data2D.createDataObject("corr", "SOFR-1B", corr_pivot))


In [161]:
one_bd = 1.0/252.0
sabr_expiries_1bd = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]

def normalvol_1bd(expiry):
    return float(0.0095*np.exp(-0.22*expiry) + 0.0026)

def nu_1bd(expiry):
    return float(0.85*np.exp(-0.35*expiry) + 0.20)

def rho_1bd(expiry):
    return float(np.clip(-0.42 + 0.06*np.log(1.0 + expiry), -0.60, -0.05))

beta_fixed = 0.60

rows = []
for e in sabr_expiries_1bd:
    rows.append([
        "SOFR-1B",
        float(e),
        float(one_bd),
        normalvol_1bd(e),
        float(beta_fixed),
        nu_1bd(e),
        rho_1bd(e),
    ])

sabr_sofr_1bd_3 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])


m = ((sabr_sofr_1bd_3["INDEX"] == "SOFR-1B") & np.isclose(sabr_sofr_1bd_3["AXIS1"], 1.0)) 

sabr_sofr_1bd_3.loc[m, "BETA"] += 0.01

for idx_name, sub in sabr_sofr_1bd_3.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        bu_objs_3.append(
            Data2D.createDataObject(
                data_type=param.lower(),
                data_convention=idx_name,
                df=pivot
            )
        )

In [162]:
value_date = "2025-09-24"

bottom_up_build_methods_sofr_2 = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr_2.append({
        "TARGET":          "SOFR-1B",
        "VALUES":          param,
        "INTERPOLATION":   "LINEAR",
        "SHIFT":           0.0,
        "VOL_DECAY_SPEED": 0.2
    })

bu_dc_3 = DataCollection(bu_objs_3)

bottom_up_sofr_sabr_4 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=bu_dc_3,
    buildMethodCollection=bottom_up_build_methods_sofr_2,
    ycModel=yc
)

print("Bottom-up 1BD-tenor SABR components:", bottom_up_sofr_sabr_4.components.keys())

Bottom-up 1BD-tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [163]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr}

ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr_4,
    vp,
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)

d = ve_bu_cap.value[1]

Overnight Caplet PV (bottom-up): ['USD', np.float64(5404.440066975963)]


In [164]:
(d-a)/0.01

np.float64(-31.00271446583065)

In [165]:
# Corr surface
corr_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]
corr_tenors   = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 30.0]

def corr_surface(expiry, tenor):
    base = 0.86
    decay_e = 0.08*np.log(1.0 + 2.0*expiry)
    decay_t = 0.10*np.log(1.0 + tenor)
    corr = base - decay_e - decay_t
    return float(np.clip(corr, 0.50, 0.90))

rows = []
for e in corr_expiries:
    for t in corr_tenors:
        rows.append([float(e), float(t), corr_surface(e, t)])

df_corr = pd.DataFrame(rows, columns=["EXPIRY","TENOR","CORR"])
df_corr["INDEX"] = "SOFR-1B"

corr_pivot = (df_corr.pivot(index="EXPIRY", columns="TENOR", values="CORR").sort_index(axis=0).sort_index(axis=1))

bu_objs_4 = []
bu_objs_4.append(Data2D.createDataObject("corr", "SOFR-1B", corr_pivot))


In [166]:
one_bd = 1.0/252.0
sabr_expiries_1bd = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]

def normalvol_1bd(expiry):
    return float(0.0095*np.exp(-0.22*expiry) + 0.0026)

def nu_1bd(expiry):
    return float(0.85*np.exp(-0.35*expiry) + 0.20)

def rho_1bd(expiry):
    return float(np.clip(-0.42 + 0.06*np.log(1.0 + expiry), -0.60, -0.05))

beta_fixed = 0.60

rows = []
for e in sabr_expiries_1bd:
    rows.append([
        "SOFR-1B",
        float(e),
        float(one_bd),
        normalvol_1bd(e),
        float(beta_fixed),
        nu_1bd(e),
        rho_1bd(e),
    ])

sabr_sofr_1bd_4 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

m = ((sabr_sofr_1bd_4["INDEX"] == "SOFR-1B") & np.isclose(sabr_sofr_1bd_4["AXIS1"], 1.0)) 

sabr_sofr_1bd_4.loc[m, "NU"] += 0.01

for idx_name, sub in sabr_sofr_1bd_4.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        bu_objs_4.append(
            Data2D.createDataObject(
                data_type=param.lower(),
                data_convention=idx_name,
                df=pivot
            )
        )

In [167]:
value_date = "2025-09-24"

bottom_up_build_methods_sofr_4 = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr_4.append({
        "TARGET":          "SOFR-1B",
        "VALUES":          param,
        "INTERPOLATION":   "LINEAR",
        "SHIFT":           0.0,
        "VOL_DECAY_SPEED": 0.2
    })

bu_dc_4 = DataCollection(bu_objs_4)

bottom_up_sofr_sabr_5 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=bu_dc_4,
    buildMethodCollection=bottom_up_build_methods_sofr_4,
    ycModel=yc
)

print("Bottom-up 1BD-tenor SABR components:", bottom_up_sofr_sabr_4.components.keys())

Bottom-up 1BD-tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [170]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr}

ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr_5,
    vp,
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)

e = ve_bu_cap.value[1]

Overnight Caplet PV (bottom-up): ['USD', np.float64(5405.55105726054)]


In [171]:
(e-a)/0.01

np.float64(80.09631399181671)

In [172]:
# Corr surface
corr_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]
corr_tenors   = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 30.0]

def corr_surface(expiry, tenor):
    base = 0.86
    decay_e = 0.08*np.log(1.0 + 2.0*expiry)
    decay_t = 0.10*np.log(1.0 + tenor)
    corr = base - decay_e - decay_t
    return float(np.clip(corr, 0.50, 0.90))

rows = []
for e in corr_expiries:
    for t in corr_tenors:
        rows.append([float(e), float(t), corr_surface(e, t)])

df_corr = pd.DataFrame(rows, columns=["EXPIRY","TENOR","CORR"])
df_corr["INDEX"] = "SOFR-1B"

corr_pivot = (df_corr.pivot(index="EXPIRY", columns="TENOR", values="CORR").sort_index(axis=0).sort_index(axis=1))

bu_objs_5 = []
bu_objs_5.append(Data2D.createDataObject("corr", "SOFR-1B", corr_pivot))


In [173]:
one_bd = 1.0/252.0
sabr_expiries_1bd = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]

def normalvol_1bd(expiry):
    return float(0.0095*np.exp(-0.22*expiry) + 0.0026)

def nu_1bd(expiry):
    return float(0.85*np.exp(-0.35*expiry) + 0.20)

def rho_1bd(expiry):
    return float(np.clip(-0.42 + 0.06*np.log(1.0 + expiry), -0.60, -0.05))

beta_fixed = 0.60

rows = []
for e in sabr_expiries_1bd:
    rows.append([
        "SOFR-1B",
        float(e),
        float(one_bd),
        normalvol_1bd(e),
        float(beta_fixed),
        nu_1bd(e),
        rho_1bd(e),
    ])

sabr_sofr_1bd_5 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

m = ((sabr_sofr_1bd_5["INDEX"] == "SOFR-1B") & np.isclose(sabr_sofr_1bd_5["AXIS1"], 1.0)) 

sabr_sofr_1bd_5.loc[m, "RHO"] += 0.01
for idx_name, sub in sabr_sofr_1bd_5.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        bu_objs_5.append(
            Data2D.createDataObject(
                data_type=param.lower(),
                data_convention=idx_name,
                df=pivot
            )
        )

In [174]:
value_date = "2025-09-24"

bottom_up_build_methods_sofr_5 = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr_5.append({
        "TARGET":          "SOFR-1B",
        "VALUES":          param,
        "INTERPOLATION":   "LINEAR",
        "SHIFT":           0.0,
        "VOL_DECAY_SPEED": 0.2
    })

bu_dc_5 = DataCollection(bu_objs_5)

bottom_up_sofr_sabr_6 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=bu_dc_5,
    buildMethodCollection=bottom_up_build_methods_sofr_5,
    ycModel=yc
)

print("Bottom-up 1BD-tenor SABR components:", bottom_up_sofr_sabr_6.components.keys())

Bottom-up 1BD-tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [175]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr}

ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr_6,
    vp,
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)

f = ve_bu_cap.value[1]

Overnight Caplet PV (bottom-up): ['USD', np.float64(5404.353156493808)]


In [176]:
(f-a)/0.01

np.float64(-39.693762681417866)